<a href="https://colab.research.google.com/github/Av1chem/Google-images-scraper/blob/master/CryptoWrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisites

In [ ]:
!pip install pycrypto requests

In [ ]:
import base64
import binascii
import json
import uuid

import requests
from Crypto.Cipher import AES
from Crypto.Protocol.KDF import PBKDF2
from Crypto.Util.py3compat import *

# Definition

In [ ]:
class RequestsWrapperForBCBSMN:
    def get(self, *args, **kwargs):
        return requests.get(*args, **kwargs)

    def post(self, url, json, *args, **kwargs):
        json["key2id"] = self.__tokens["key2id"]
        return self.__getDecrypted(
            requests.post(
                url=url,
                json=self.__getEncrypted(json),
                headers={"uitxnid": f"WEB_v3.0_{uuid.uuid4()}"},
            ).json()["message"]
        )

    def __init__(self, *args, **kwargs):
        self.__tokens = requests.get(
            "https://mobilemember.bluecrossma.com/dglwebapi1/mobilekeyservice/v1/gettokens"
        ).json()

    def __enter__(self):
        return self

    def __exit__(self, *args, **kwargs):
        del self

    ####
    # Encryption/Decryption Logic
    ####
    # Missing pad function from Crypto
    # https://github.com/pycrypto/pycrypto/blob/master/lib/Crypto/Util/Padding.py
    # TODO: move to its own file
    def __pad(self, data_to_pad, block_size, style="pkcs7"):
        """Apply standard padding.
        :Parameters:
          data_to_pad : byte string
            The data that needs to be padded.
          block_size : integer
            The block boundary to use for padding. The output length is guaranteed
            to be a multiple of ``block_size``.
          style : string
            Padding algorithm. It can be *'pkcs7'* (default), *'iso7816'* or *'x923'*.
        :Return:
          The original data with the appropriate padding added at the end.
        """

        padding_len = block_size - len(data_to_pad) % block_size
        if style == "pkcs7":
            padding = bchr(padding_len) * padding_len
        elif style == "x923":
            padding = bchr(0) * (padding_len - 1) + bchr(padding_len)
        elif style == "iso7816":
            padding = bchr(128) + bchr(0) * (padding_len - 1)
        else:
            raise ValueError("Unknown padding style")
        return data_to_pad + padding

    def __hexParse(self, str_to_parse):
        return binascii.a2b_hex(str_to_parse)

    def __json_stringify(self, json_payload):
        return json.dumps(json_payload, separators=(",", ":")).encode()

    def __getEncrypted(self, payload):
        payload_json = self.__json_stringify(payload)
        payload_padded = self.__pad(payload_json, AES.block_size, style="pkcs7")
        key1salt_hex = self.__hexParse(self.__tokens["key1salt"])
        key1iv_hex = self.__hexParse(self.__tokens["key1iv"])
        key1phrase = self.__tokens["key1phrase"]

        r = PBKDF2(key1phrase, key1salt_hex, count=10000)
        cipher = AES.new(r, AES.MODE_CBC, key1iv_hex)
        encrypted_payload = cipher.encrypt(payload_padded)
        a = base64.b64encode(encrypted_payload)

        return {"message": a.decode(), "key1id": self.__tokens["key1id"]}

    def __getDecrypted(self, message):
        key2phrase = self.__tokens["key2phrase"]
        key2salt_hex = self.__hexParse(self.__tokens["key2salt"])
        key2iv_hex = self.__hexParse(self.__tokens["key2iv"])
        message_base64decoded = base64.b64decode(message)

        n = PBKDF2(key2phrase, key2salt_hex, count=10000)
        cipher = AES.new(n, AES.MODE_CBC, key2iv_hex)
        i = cipher.decrypt(message_base64decoded)
        i = i.decode("utf-8")
        i = "".join(c for c in i if c.isprintable())
        return json.loads(i)

# Usage

In [ ]:
arbitrary_network = {
    "id": 311005007,
    "name": "DENTAL BLUE PPO",
    "planIndicator": False,
}

arbitrary_payload = {
    "geoLocation": "42.5562716,-72.5167854",
    "limit": 20,
    "page": 1,
    "radius": 25,
    "networkId": arbitrary_network["id"],
    "searchForTH": False,
    "useridin": "undefined",
    "fadVendorMemberNumber": None,
}

In [ ]:
with RequestsWrapperForBCBSMN() as rw:

    resp = rw.post(
        "https://bcbsma-prod.apigee.net/member/web/v1/vitalscommon/searchbyproviders",
        json=arbitrary_payload,
    )
    print(json.dumps(resp, indent=4))